# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG =  or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 100

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
# 'Random': or_suite.agents.rl.random.randomAgent(),
# 'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
# 'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[2.042 1.957 2.034 1.924 1.975 2.035 2.041 2.013 1.964 2.016]
 [2.999 3.066 3.013 3.03  3.02  3.081 2.952 2.974 3.031 2.914]
 [3.977 3.951 4.032 3.958 4.07  4.009 4.04  3.997 4.016 4.04 ]] [[1.088236 0.897151 1.058844 0.916224 0.950375 1.031775 1.001319 1.044831
  0.992704 1.089744]
 [2.096999 2.017644 2.136831 2.1771   2.0376   1.938439 1.929696 1.963324
  1.966039 1.914604]
 [2.794471 2.894599 3.258976 2.892236 3.1611   2.970919 2.9084   2.846991
  3.181744 3.0124  ]]
Mean and variance endomwnets:
[[1.971 2.033 1.979 2.008 2.014 2.004 1.991 1.993 1.992 1.975]
 [2.924 2.959 2.98  2.956 3.006 2.944 2.989 2.94  3.054 3.065]
 [3.927 3.978 3.9   3.978 4.006 3.943 4.05  4.04  4.033 3.999]] [[0.986159 1.107911 0.992559 1.053936 0.977804 0.945984 0.910919 0.966951
  0.993936 1.034375]
 [1.820224 1.961319 2.0716   1.980064 2.063964 1.834864 2.004879 1.8064
  2.201084 2.142775]
 [3.181671 2.963516 2.962    3.133516 2.985964 3.113751 3.1035   3.0384
  3.053911 3.2

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

Guardrail-0.5
Lower and Upper Solutions:
[[0.      0.18736]
 [0.      0.18736]
 [0.21893 0.     ]]
[[0.      0.27401]
 [0.      0.27401]
 [0.32018 0.     ]]
Writing to file data.csv
Guardrail-0.3
Lower and Upper Solutions:
[[ 0.       0.18794]
 [-0.       0.18794]
 [ 0.21988  0.     ]]
[[0.      0.37678]
 [0.      0.37678]
 [0.44081 0.     ]]
Writing to file data.csv
Guardrail-0.25
Lower and Upper Solutions:
[[0.      0.18831]
 [0.      0.18831]
 [0.21918 0.     ]]
[[0.      0.43028]
 [0.      0.43028]
 [0.50081 0.     ]]
Writing to file data.csv
        Algorithm  Reward      Time     Space  Efficiency  Hindsight Envy  \
0   Guardrail-0.5    -inf  5.196850 -14833.05 -166.938827     -192.473320   
1   Guardrail-0.3    -inf  4.957541 -12641.32 -164.497911     -216.268068   
2  Guardrail-0.25    -inf  5.198281 -12493.16 -174.906971     -212.900625   

   Counterfactual Envy  
0           -42.465574  
1           -69.251330  
2           -72.112810  


In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)